In [177]:
import numpy as np
import pandas as pd

In [216]:
# read in dataset
data=pd.read_csv('Food Security Indicators-raw.csv')
data.head()

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,FS,Suite of Food Security Indicators,26,Brunei Darussalam,6121,Value,21010,Average dietary energy supply adequacy (percen...,20002002,2000-2002,%,123,F,FAO estimate,NaN
1,FS,Suite of Food Security Indicators,26,Brunei Darussalam,6121,Value,21010,Average dietary energy supply adequacy (percen...,20012003,2001-2003,%,125,F,FAO estimate,NaN
2,FS,Suite of Food Security Indicators,26,Brunei Darussalam,6121,Value,21010,Average dietary energy supply adequacy (percen...,20022004,2002-2004,%,127,F,FAO estimate,NaN
3,FS,Suite of Food Security Indicators,26,Brunei Darussalam,6121,Value,21010,Average dietary energy supply adequacy (percen...,20032005,2003-2005,%,127,F,FAO estimate,NaN
4,FS,Suite of Food Security Indicators,26,Brunei Darussalam,6121,Value,21010,Average dietary energy supply adequacy (percen...,20042006,2004-2006,%,127,F,FAO estimate,NaN


In [217]:
# drop useless or duplicated columns
data=data.drop(['Domain Code', 'Domain','Area Code (FAO)', 'Year Code','Element Code','Element',
            'Item Code','Unit', 'Flag',
              'Flag Description','Note'], axis=1)
data.head()

,Area,Item,Year,Value
0,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2000-2002,123
1,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2001-2003,125
2,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2002-2004,127
3,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2003-2005,127
4,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2004-2006,127


In [219]:
# Compute missing values
data.isnull().sum()

Area       0
Item       0
Year       0
Value    124
dtype: int64

In [220]:
# Impute missing values to 0
data.fillna(0,inplace=True)
print(data.isnull().sum())

Area     0
Item     0
Year     0
Value    0
dtype: int64


In [221]:
# Record Items with inconsistent forms of Year column
print('These are Items with inconsistent form of Year:')
n=1
for i in range(len(data)):
    if data.iloc[i,0]=='Brunei Darussalam'and data.iloc[i,2]=='2000-2002':
        print(n,data.iloc[i,1])
        n+=1
        pass
    pass


These are Items with inconsistent form of Year:
1 Average dietary energy supply adequacy (percent) (3-year average)
2 Average value of food production (constant 2004-2006 I$/cap) (3-year average)
3 Dietary energy supply used in the estimation of prevalence of undernourishment (kcal/cap/day) (3-year average)
4 Share of dietary energy supply derived from cereals, roots and tubers (kcal/cap/day) (3-year average)
5 Average protein supply (g/cap/day) (3-year average)
6 Average supply of protein of animal origin (g/cap/day) (3-year average)
7 Prevalence of undernourishment (percent) (3-year average)
8 Number of people undernourished (million) (3-year average)
9 Cereal import dependency ratio (percent) (3-year average)
10 Percent of arable land equipped for irrigation (percent) (3-year average)
11 Value of food imports in total merchandise exports (percent) (3-year average)
12 Average fat supply (g/cap/day) (3-year average)


In [222]:
# Formatting 'Year' column
for i in range(len(data)):
    if len(data.iloc[i,2])==9:
        data.iloc[i,2]=int((int(data.iloc[i,2][0:4])+int(data.iloc[i,2][5:]))/2)
        pass
    elif len(data.iloc[i,2])==4:
        data.iloc[i,2]=int(data.iloc[i,2])
        pass
    else:
        print('There is an error in Year column')
        pass
    pass
data.head()

,Area,Item,Year,Value
0,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2001,123
1,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2002,125
2,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2003,127
3,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2004,127
4,Brunei Darussalam,Average dietary energy supply adequacy (percen...,2005,127


In [223]:
# Formatting 'Value' column
data['Value'] = data.Value.apply(str)
index = data[data['Value'].str.contains('<'or'>')].index
for i in index:
    data.iloc[i,3] = data.iloc[i,3][1:]
    pass
data['Value'] = pd.to_numeric(data['Value'], downcast='float')

In [224]:
# values in 'Item' column contains variables we need to analysis
# should be transferred to key:value format
print(len(data.Item.unique()))     #compute variable numbers
# Transferred Item
from pandas.core.reshape.reshape import pivot
data=data.pivot_table(index=['Area','Year'],columns='Item',values='Value')
data=data.reset_index()
data.head()

42


Item,Area,Year,Average dietary energy requirement (kcal/cap/day),Average dietary energy supply adequacy (percent) (3-year average),Average fat supply (g/cap/day) (3-year average),Average protein supply (g/cap/day) (3-year average),Average supply of protein of animal origin (g/cap/day) (3-year average),Average value of food production (constant 2004-2006 I$/cap) (3-year average),Cereal import dependency ratio (percent) (3-year average),Coefficient of variation of habitual caloric consumption distribution (real number),...,Prevalence of anemia among women of reproductive age (15-49 years),Prevalence of exclusive breastfeeding among infants 0-5 months of age,Prevalence of low birthweight (percent),Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average),Prevalence of obesity in the adult population (18 years and older),Prevalence of severe food insecurity in the total population (percent) (3-year average),Prevalence of undernourishment (percent) (3-year average),Rail lines density (total route in km per 100 square km of land area),"Share of dietary energy supply derived from cereals, roots and tubers (kcal/cap/day) (3-year average)",Value of food imports in total merchandise exports (percent) (3-year average)
0,Brunei Darussalam,2000,2323.0,NaN,NaN,NaN,NaN,NaN,NaN,0.21,...,17.100000,NaN,10.3,NaN,7.2,NaN,NaN,NaN,NaN,NaN
1,Brunei Darussalam,2001,2329.0,123.0,75.699997,87.400002,45.400002,81.0,100.000000,0.21,...,16.600000,NaN,9.5,NaN,7.5,NaN,2.6,NaN,50.0,5.0
2,Brunei Darussalam,2002,2337.0,125.0,78.699997,89.699997,47.700001,85.0,99.599998,0.21,...,16.299999,NaN,9.2,NaN,7.9,NaN,2.5,NaN,48.0,4.0
3,Brunei Darussalam,2003,2345.0,127.0,81.699997,90.300003,48.299999,86.0,95.699997,0.21,...,16.000000,NaN,9.1,NaN,8.2,NaN,2.5,NaN,47.0,4.0
4,Brunei Darussalam,2004,2352.0,127.0,81.000000,88.599998,47.299999,76.0,93.800003,0.21,...,15.700000,NaN,9.2,NaN,8.6,NaN,2.5,NaN,46.0,4.0


In [225]:
#combine two dataframes
df=pd.read_csv('ASEAN__Consumer Price Indices-cleaned.csv')
data=pd.merge(data,df,on=['Area','Year'],sort=True)
data.head()

,Area,Year,Average dietary energy requirement (kcal/cap/day),Average dietary energy supply adequacy (percent) (3-year average),Average fat supply (g/cap/day) (3-year average),Average protein supply (g/cap/day) (3-year average),Average supply of protein of animal origin (g/cap/day) (3-year average),Average value of food production (constant 2004-2006 I$/cap) (3-year average),Cereal import dependency ratio (percent) (3-year average),Coefficient of variation of habitual caloric consumption distribution (real number),...,Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average),Prevalence of obesity in the adult population (18 years and older),Prevalence of severe food insecurity in the total population (percent) (3-year average),Prevalence of undernourishment (percent) (3-year average),Rail lines density (total route in km per 100 square km of land area),"Share of dietary energy supply derived from cereals, roots and tubers (kcal/cap/day) (3-year average)",Value of food imports in total merchandise exports (percent) (3-year average),"Consumer Prices, Food Indices (2015 = 100)","Consumer Prices, General Indices (2015 = 100)",Food price inflation
0,Brunei Darussalam,2000,2323.0,NaN,NaN,NaN,NaN,NaN,NaN,0.21,...,NaN,7.2,NaN,NaN,NaN,NaN,NaN,89.312189,95.142982,NaN
1,Brunei Darussalam,2001,2329.0,123.0,75.699997,87.400002,45.400002,81.0,100.000000,0.21,...,NaN,7.5,NaN,2.6,NaN,50.0,5.0,89.911821,95.709930,0.671642
2,Brunei Darussalam,2002,2337.0,125.0,78.699997,89.699997,47.700001,85.0,99.599998,0.21,...,NaN,7.9,NaN,2.5,NaN,48.0,4.0,90.286058,93.494272,0.416298
3,Brunei Darussalam,2003,2345.0,127.0,81.699997,90.300003,48.299999,86.0,95.699997,0.21,...,NaN,8.2,NaN,2.5,NaN,47.0,4.0,90.033759,93.743590,-0.279126
4,Brunei Darussalam,2004,2352.0,127.0,81.000000,88.599998,47.299999,76.0,93.800003,0.21,...,NaN,8.6,NaN,2.5,NaN,46.0,4.0,90.485887,94.530500,0.503782


In [226]:
#split into different datasets by Area
df_Brunei=data[data['Area'].isin(['Brunei Darussalam'])].iloc[:,1:]
df_Cambodia=data[data['Area'].isin(['Cambodia'])].iloc[:,1:]
df_Indonesia=data[data['Area'].isin(['Indonesia'])].iloc[:,1:]
df_Lao=data[data['Area'].isin(["Lao People's Democratic Republic"])].iloc[:,1:]
df_Malaysia=data[data['Area'].isin(['Malaysia'])].iloc[:,1:]
df_Myanmar=data[data['Area'].isin(['Myanmar'])].iloc[:,1:]
df_Philippines=data[data['Area'].isin(['Philippines'])].iloc[:,1:]
df_Singapore=data[data['Area'].isin(['Singapore'])].iloc[:,1:]
df_Thailand=data[data['Area'].isin(['Thailand'])].iloc[:,1:]
df_VietNam=data[data['Area'].isin(['Viet Nam'])].iloc[:,1:]


In [228]:
#save data
data.to_csv(r'ASEAN_Food Security Indicators-cleaned.csv',index=False)
df_Brunei.to_csv(r'Brunei_Food Security Indicators.csv',index=False)
df_Cambodia.to_csv(r'Cambodia_Food Security Indicators.csv',index=False)
df_Indonesia.to_csv(r'Indonesia_Food Security Indicators.csv',index=False)
df_Lao.to_csv(r'Lao_Food Security Indicators.csv',index=False)
df_Malaysia.to_csv(r'Malaysia_Food Security Indicators.csv',index=False)
df_Myanmar.to_csv(r'Myanmar_Food Security Indicators.csv',index=False)
df_Philippines.to_csv(r'Philippines_Food Security Indicators.csv',index=False)
df_Singapore.to_csv(r'Singapore_Food Security Indicators.csv',index=False)
df_Thailand.to_csv(r'Thailand_Food Security Indicators.csv',index=False)
df_VietNam.to_csv(r'VietNam_Food Security Indicators.csv',index=False)

